# Gift Recommendation System

## Files You need to install.

In [1]:
#pip install pandas
#pip install requests
#pip install bs4
#pip install nltk
#pip install autocorrect
#nltk.download('punkt')
#nltk.download('wordnet')
#pip install tensorflow
#pip install keras
#pip install string

## Import All Basic Files.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

## Import Text Preprocessing Files.

In [2]:
from nltk.stem import WordNetLemmatizer as WNL
from nltk import word_tokenize
import nltk
from nltk.corpus import wordnet 
from autocorrect import Speller
from nltk.corpus import stopwords
import re

## Import Files required to make recommendation system.

In [3]:
from keras.utils import pad_sequences
import tensorflow as tf
import string
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
Table=pd.read_csv("Reviews.csv")
Table.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


## Perform EDA(Exploratory Data Analysis).

In [6]:
Table.describe()

,Id,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time
count,568454.000000,568454.000000,568454.00000,568454.000000,5.684540e+05
mean,284227.500000,1.743817,2.22881,4.183199,1.296257e+09
std,164098.679298,7.636513,8.28974,1.310436,4.804331e+07
min,1.000000,0.000000,0.00000,1.000000,9.393408e+08
25%,142114.250000,0.000000,0.00000,4.000000,1.271290e+09
50%,284227.500000,0.000000,1.00000,5.000000,1.311120e+09
75%,426340.750000,2.000000,2.00000,5.000000,1.332720e+09
max,568454.000000,866.000000,923.00000,5.000000,1.351210e+09


In [7]:
Table.shape

(568454, 10)

## Check Null in Dataset has their are few of them and we have enough data so we  can easy remove those row.

In [8]:
Table.isnull().sum()

Id                         0
ProductId                  0
UserId                     0
ProfileName               26
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64

In [9]:
Table.dropna(inplace=True)

In [10]:
print("Null Value:")
Table.isnull().sum()

Null Value:


Id                        0
ProductId                 0
UserId                    0
ProfileName               0
HelpfulnessNumerator      0
HelpfulnessDenominator    0
Score                     0
Time                      0
Summary                   0
Text                      0
dtype: int64

In [11]:
Table.shape

(568401, 10)

## Check For Duplicates and Remove them.

In [12]:
Table["Id"].duplicated().sum()


0

In [13]:
Duplicate=[(i,Table["Id"].duplicated().sum()) for i in Table.columns]
print("Duplicate Value:\n")
_=[print(i,"=",j) for i,j in Duplicate]


Duplicate Value:

Id = 0
ProductId = 0
UserId = 0
ProfileName = 0
HelpfulnessNumerator = 0
HelpfulnessDenominator = 0
Score = 0
Time = 0
Summary = 0
Text = 0


## Remove Columns That is Not Required.

In [14]:
Table.drop(columns=["UserId","Time","ProfileName"],axis=1,inplace=True)

In [15]:
Table.head()

,Id,ProductId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Summary,Text
0,1,B001E4KFG0,1,1,5,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,0,0,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,1,1,4,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,3,3,2,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,0,0,5,Great taffy,Great taffy at a great price. There was a wid...


In [16]:
Table.set_index("Id",inplace=True)

In [17]:
Table.head()

,ProductId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Summary,Text
Id,,,,,,
1,B001E4KFG0,1,1,5,Good Quality Dog Food,I have bought several of the Vitality canned d...
2,B00813GRG4,0,0,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
3,B000LQOCH0,1,1,4,"""Delight"" says it all",This is a confection that has been around a fe...
4,B000UA0QIQ,3,3,2,Cough Medicine,If you are looking for the secret ingredient i...
5,B006K2ZZ7K,0,0,5,Great taffy,Great taffy at a great price. There was a wid...


## Table with Score 4 and more.

In [18]:
Table=Table[Table["Score"]>=4]

## Display unique Number of users who found the review helpful and get Table with HelpfulnessNumerator more than 20.


In [19]:
Table["HelpfulnessNumerator"].unique()

array([  1,   0,   4,   2,  19,  13,   9,   3,  15,   5,  17,   8,   6,
         7,  43,  14,  11,  27,  12,  10,  16,  23, 165,  45,  30,  32,
        24,  22,  20,  59,  21,  29, 112, 107,  51,  88,  35,  25,  42,
        70,  86,  48,  36,  34,  18,  53,  40,  26,  31,  65, 128,  97,
        82,  37,  28, 202,  47,  69,  56,  39, 111,  60,  66,  44,  91,
        38, 115,  93, 129,  58,  83,  55,  62,  50,  33, 135,  71,  73,
       122,  90, 203,  64, 398, 206, 133,  41,  84,  63,  95,  57,  72,
        52,  54,  96,  49, 177,  80, 137,  87, 264,  68,  77,  85, 258,
        75, 119,  61,  89, 131, 125, 536,  78, 559, 472, 232, 294,  79,
        67,  46, 154, 105, 117, 349, 130, 454, 172, 116, 179, 157, 170,
       106, 219,  94, 110,  76,  92, 123, 413, 139, 866, 233,  99,  81,
       366, 124, 138,  74, 121, 155, 113, 103, 356, 439, 193, 466, 429,
       120, 538, 178, 257, 212, 205, 108, 153, 278, 164, 158, 126, 100,
       406, 290, 166, 150, 488, 218, 143, 148, 118, 145, 209, 10

In [20]:
Table=Table[Table["HelpfulnessNumerator"]>10]

## Display unique Number of users who indicated whether they found the review helpful or not more than 10.

In [21]:
Table["HelpfulnessDenominator"].unique()

array([ 19,  13,  15,  47,  17,  14,  27,  12,  21,  18,  23,  11, 168,
        46,  30,  36,  25,  16,  29,  20,  22,  64,  24,  33, 112, 108,
        51,  89,  35,  42,  74,  49,  34,  58,  28,  41,  26,  31,  65,
       130, 100,  84,  38,  40, 219,  72,  56,  63, 128,  79,  66,  44,
        94,  59, 117, 134,  39,  92,  57,  53,  37,  45, 135,  71,  43,
        77,  83, 123,  50,  93, 212,  32,  52, 401, 209,  86,  99,  60,
        95,  73,  54,  61,  70, 180,  91, 138,  55, 163, 270,  85,  78,
       120,  62, 133,  88, 137,  48, 539,  81, 562, 477, 235, 139,  87,
        69, 298, 156, 110, 144,  80,  68,  75, 356, 462, 174, 111,  76,
       182, 101, 171,  97, 187, 121,  67, 263, 131, 165,  96, 416, 106,
       878, 238, 115, 150, 371, 125, 149, 122,  90, 185, 114, 124, 105,
        98, 365, 452, 197, 488, 118, 436, 176, 132, 544, 184, 274, 240,
        82, 229, 284, 158, 127, 415, 296, 172, 496, 220, 152, 147, 102,
       143, 214, 107, 230, 282, 200, 103, 322, 162, 140, 228, 19

In [22]:
Tabale=Table[Table["HelpfulnessDenominator"]>10]

In [23]:
Table.shape

(10501, 6)

## Convert Productid to Product_Name Using BeautifulSoup

In [24]:
def get_product_name(product_id):
    url = f"https://www.amazon.com/dp/{product_id}"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    product_title = soup.find('span', id='productTitle')
    if product_title:
        return product_title.text.strip()
    else:
        return "Product not found"
Table["Product_Name"]=Table["ProductId"].apply(get_product_name)

## Remove Product not found Row.

In [26]:
Table=Table[Table["Product_Name"]!="Product not found"]

In [27]:
Table.shape

(1337, 7)

In [28]:
Table.head()

,ProductId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Summary,Text,Product_Name
Id,,,,,,,
1258,B001FA1L7A,13,15,5,"The true, the classic custard for authentic, y...","After you've made the customary, authentic Eng...","Bird's Custard Powder, 10.6-Ounce Canisters (P..."
1573,B001CWZXIY,13,13,5,High Quality My Dog Enjoys and My Budget Does ...,My doggie really likes Newman's Organic better...,Healthy & Holistic Healthy Weight Formula For ...
1574,B001CWZXIY,12,12,5,This is great dog food!,"A few years back, prompted by the death of one...",Healthy & Holistic Healthy Weight Formula For ...
2001,B001E5E29A,11,14,5,Awesome,I usually make pancakes from scratch and have ...,Stonewall Kitchen Farmhouse Pancake & Waffle M...
3307,B005K4Q1VI,24,25,5,Really Good Hot Cocoa - Highly Recommended,These Grove Square Hot Cocoa flavors are by fa...,"Grove Square Hot Cocoa Pods, Milk Chocolate, S..."


## Concate Summary and Text to easy out Calculation.

In [29]:
Table["Summary_Text"]=Table["Summary"]+Table["Text"]

## Remove Column That is not Required.

In [30]:
Table.drop(columns=["ProductId","HelpfulnessNumerator","HelpfulnessDenominator","Score","Summary","Text"],axis=1,inplace=True)

## Pre-processing Text.

## Now we have to remove html Tags.

In [31]:
set(str(Table["Summary_Text"]).split())

{'!Being',
 '-',
 '...',
 '1258',
 '1337,',
 '1573',
 '1574',
 '2001',
 '3307',
 '567029',
 '567030',
 '567254',
 '567530',
 '568160',
 'AwesomeI',
 'Budget',
 'Cocoa',
 'Does',
 'Dog',
 'Enjoys',
 'Excellent',
 'FabulousMy',
 'French',
 'Good',
 'High',
 'Highly',
 'Hot',
 'I',
 'Id',
 'Illy',
 'Just',
 'Length:',
 'My',
 'Name:',
 'Quality',
 'Really',
 'RecommendedThes...',
 'So',
 'Summary_Text,',
 'The',
 'This',
 'a',
 'about',
 'amount',
 'an...',
 'and',
 'authentic,',
 'back,',
 'best',
 'by',
 'classic',
 "coffeeI'm",
 'compa...',
 'considerable',
 'custard',
 'dog',
 'dtype:',
 'espresso.After...',
 'far',
 'far,',
 'favorite',
 'few',
 'food!A',
 'for',
 'from',
 'grandmother',
 'great',
 'grew',
 'is',
 'kno...',
 'like',
 'make',
 'my',
 'o...',
 'object',
 'pancakes',
 'pre-ground',
 'promp...',
 'remember',
 'scratch',
 'spent',
 'still',
 'talking',
 'tasting',
 'the',
 'true,',
 'up',
 'usually',
 'valueI',
 'y...',
 'years'}

In [35]:
def remove_html_tag(text):
    pattern=re.compile("<.*?>")
    return pattern.sub(r'',text)
Table["Summary_Text"]=Table["Summary_Text"].apply(remove_html_tag)

In [36]:
set(str(Table["Summary_Text"]).split())

{'!Being',
 '-',
 '...',
 '1258',
 '1337,',
 '1573',
 '1574',
 '2001',
 '3307',
 '567029',
 '567030',
 '567254',
 '567530',
 '568160',
 'AwesomeI',
 'Budget',
 'Cocoa',
 'Does',
 'Dog',
 'Enjoys',
 'Excellent',
 'FabulousMy',
 'French',
 'Good',
 'High',
 'Highly',
 'Hot',
 'I',
 'Id',
 'Illy',
 'Just',
 'Length:',
 'My',
 'Name:',
 'Quality',
 'Really',
 'RecommendedThes...',
 'So',
 'Summary_Text,',
 'The',
 'This',
 'a',
 'about',
 'amount',
 'an...',
 'and',
 'authentic,',
 'back,',
 'best',
 'by',
 'classic',
 "coffeeI'm",
 'compa...',
 'considerable',
 'custard',
 'dog',
 'dtype:',
 'espresso.After...',
 'far',
 'far,',
 'favorite',
 'few',
 'food!A',
 'for',
 'from',
 'grandmother',
 'great',
 'grew',
 'is',
 'kno...',
 'like',
 'make',
 'my',
 'o...',
 'object',
 'pancakes',
 'pre-ground',
 'promp...',
 'remember',
 'scratch',
 'spent',
 'still',
 'talking',
 'tasting',
 'the',
 'true,',
 'up',
 'usually',
 'valueI',
 'y...',
 'years'}

## Check and Correct spelling of data.

In [38]:
spell = Speller(fast=True)
Table["Summary_Text"]=Table["Summary_Text"].apply(lambda txt: ''.join(spell(txt)))

## Now we are going to do Lemmatization to reduce word so that calculation becomes easy.
Lemmatization is a text preprocessing technique that reduces words to their base form, or lemma, to make them easier to analyze.

## Remove Stop words as it is not required.

In [39]:
get_wordnet_pos=lambda treebank_tag: wordnet.ADJ if treebank_tag[0]=="J" else wordnet.VERB if treebank_tag[0]=="V" else wordnet.ADV if treebank_tag[0]=="R" else wordnet.NOUN 

In [40]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WNL()
    def __call__(self, articles):
        lem=list()
        return [self.wnl.lemmatize(t.lower(),pos=get_wordnet_pos(nltk.pos_tag(list(t.lower()))[0][1])) for t in word_tokenize(articles) if t not in set(stopwords.words('english')) and (t not in string.punctuation) and (t.isnumeric()==False)]

## Tokenization (Tokenization is the process of breaking text into smaller units, called tokens, that are usually words or subwords.) of Text.

In [41]:
tok=tf.keras.preprocessing.text.Tokenizer(
    num_words=None,
    filters=None,
    lower=False,
    split=' ',
    char_level=False,
    oov_token="<nothing>",
    analyzer=LemmaTokenizer(),
)

## Vectorization (Vectorization is a process in Natural Language Processing (NLP) that converts text data into a numerical format that machine learning algorithms can understand.) of Text.

In [42]:
tok.fit_on_texts(Table["Summary_Text"])

In [43]:
tok.word_index

{'<nothing>': 1,
 'i': 2,
 "'s": 3,
 "n't": 4,
 'taste': 5,
 'like': 6,
 'it': 7,
 'the': 8,
 'product': 9,
 'make': 10,
 'good': 11,
 'great': 12,
 'one': 13,
 'coffee': 14,
 "''": 15,
 '``': 16,
 'use': 17,
 'flavor': 18,
 'food': 19,
 'get': 20,
 'also': 21,
 "'ve": 22,
 'this': 23,
 'tea': 24,
 'would': 25,
 'love': 26,
 '...': 27,
 'time': 28,
 'best': 29,
 'much': 30,
 'really': 31,
 'water': 32,
 'eat': 33,
 'oil': 34,
 'tried': 35,
 'cup': 36,
 'dog': 37,
 'well': 38,
 'little': 39,
 'price': 40,
 'chocolate': 41,
 'sugar': 42,
 "'m": 43,
 'better': 44,
 'say': 45,
 'a': 46,
 'try': 47,
 'coconut': 48,
 'bag': 49,
 'year': 50,
 'find': 51,
 'amazon': 52,
 'even': 53,
 'brand': 54,
 'found': 55,
 'take': 56,
 'want': 57,
 'first': 58,
 'thing': 59,
 'give': 60,
 '--': 61,
 'add': 62,
 'my': 63,
 'come': 64,
 'buy': 65,
 'they': 66,
 'used': 67,
 'if': 68,
 'milk': 69,
 'go': 70,
 'way': 71,
 'salt': 72,
 'work': 73,
 'day': 74,
 'two': 75,
 'using': 76,
 'know': 77,
 'store': 78

In [44]:
tok.document_count

1337

In [45]:
x=tok.texts_to_sequences(Table["Summary_Text"])
x

[[8,
  451,
  1277,
  1618,
  1352,
  459,
  1114,
  6240,
  22,
  10,
  6241,
  1352,
  1114,
  3131,
  1168,
  3,
  1618,
  161,
  297,
  57,
  70,
  122,
  1428,
  6242,
  1219,
  92,
  2,
  3132,
  333,
  269,
  754,
  1168,
  3,
  1618,
  1220,
  168,
  923,
  168,
  6243,
  652,
  207,
  2,
  960,
  2729,
  1115,
  6244,
  534,
  1353,
  6245,
  1014,
  2,
  21,
  1619,
  291,
  1168,
  3,
  1618,
  161,
  444,
  2730,
  489,
  1354,
  148,
  10,
  115,
  3131,
  9,
  197,
  6246,
  414,
  489,
  2,
  1619,
  6247,
  806,
  379,
  2026,
  1168,
  3,
  1618,
  198,
  6248,
  3131,
  148,
  10,
  1278,
  1168,
  3,
  1618,
  161,
  3661,
  260,
  362,
  4553,
  961,
  292,
  924,
  696,
  388,
  514,
  293,
  4553,
  961,
  292,
  3133,
  99,
  1730,
  362,
  197,
  2027,
  807,
  62,
  2731,
  2425,
  16,
  1520,
  15,
  107,
  1521,
  2,
  155,
  102,
  9,
  755,
  244,
  333,
  52,
  104,
  40,
  923,
  291,
  859,
  147,
  4554,
  70],
 [118,
  96,
  63,
  37,
  2426,
  63,
  2

In [46]:
x=pad_sequences(x,padding="post")

In [47]:
gift=pd.DataFrame(cosine_similarity(x))

## Save the File.

In [49]:
#gift.to_csv("Similarity_score.csv",index=False)
#Table.to_csv("Product_name.csv",index=False)

In [13]:
gift=np.array(pd.read_csv("Similarity_score.csv"))
Table=pd.read_csv("Product_name.csv")

## We Are Going to Make Content Based Recommender System 
(Content-based recommender systems are a type of algorithm that suggests items to users based on their past interactions with similar items. These systems analyze the content of items, such as movies, articles, or products, and recommend similar items to users).

In [14]:
gift

array([[1.        , 0.24597845, 0.3534969 , ..., 0.071919  , 0.2045874 ,
        0.08028765],
       [0.24597845, 1.        , 0.23604617, ..., 0.01846775, 0.1362606 ,
        0.16469083],
       [0.3534969 , 0.23604617, 1.        , ..., 0.06014067, 0.20538423,
        0.0889423 ],
       ...,
       [0.071919  , 0.01846775, 0.06014067, ..., 1.        , 0.05876854,
        0.06251657],
       [0.2045874 , 0.1362606 , 0.20538423, ..., 0.05876854, 1.        ,
        0.11872702],
       [0.08028765, 0.16469083, 0.0889423 , ..., 0.06251657, 0.11872702,
        1.        ]])

In [30]:
Table.head()

,Product_Name,Summary_Text
0,"Bird's Custard Powder, 10.6-Ounce Canisters (P...","The true, the classic custard for authentic, d..."
1,Healthy & Holistic Healthy Weight Formula For ...,High Quality My Dog Enjoys and My Budget Does ...
2,Healthy & Holistic Healthy Weight Formula For ...,"This is great dog food!A few years back, promp..."
3,Stonewall Kitchen Farmhouse Pancake & Waffle M...,Awesome usually make pancake from scratch and ...
4,"Grove Square Hot Cocoa Pods, Milk Chocolate, S...",Really Good Hot Cocoa - Highly RecommendedThes...


In [29]:
Table.index=np.arange(0,len(Table))

In [69]:
def recommend(product):
    Index=Table[Table["Product_Name"].str.contains(product)].index[0]
    List=sorted(enumerate(gift[Index]),reverse=True,key=lambda x:x[1])[1:6]
    for i in List:
        print(Table.loc[i[0],"Product_Name"])
        print()
    

In [90]:
recommend("Powdered Peanut Butter")

PB2 Powdered Peanut Butter,6.5 oz

MIO Fruit Punch, 1.6-Ounce (Pack of 4)

Vital Gluten Flour, 1 lb.

Eden Organic Black Beans, 15 oz Can (12-Pack Case), No Salt Added, Non-GMO, U.S Grown, Heat and Serve, Macrobiotic, Turtle Beans, Frijol Negro, Caviar Criollo

Numi Tea Velvet Flowering Tea Chest 12-count chest



In [91]:
recommend("Kettle Brand Potato Chips")

Kettle Brand Potato Chips, Backyard Barbeque Kettle Chips, Snack Bag 1 Oz (Pack of 72)

Kettle Brand Potato Chips, Honey Dijon Bags, 2 Ounce (Pack of 24)

Bob's Red Mill Gluten Free All Purpose Baking Flour, 44 Oz (4 Pack)

Jamaica Blue Mountain Coffee, Certified 100% Pure, Roasted Beans in a 1lb Bag

Navitas Organics Maca Powder, 16 oz. Bag, 90 Servings — Organic, Non-GMO, Low Temp-Dried, Gluten-Free

